# 이미지 2만장 생성(영상마다 처리해줌)

In [ ]:
import os
import cv2

# 영상 파일들이 있는 폴더 경로
video_folder = '/content/drive/MyDrive/a/주행영상1'

# 프레임 단위로 이미지 저장할 디렉토리
output_directory = '/content/drive/MyDrive/a/주행영상1_이미지처리'
# 디렉토리가 없으면 생성
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# 폴더 내의 모든 영상 파일에 대해 작업 수행
for filename in os.listdir(video_folder):
    if filename.endswith('.MP4'):  # 영상 파일 확장자를 변경하세요 (ex: '.mp4')
        video_path = os.path.join(video_folder, filename)

        # 영상 파일 열기
        cap = cv2.VideoCapture(video_path)

        # 프레임 수 및 영상 크기 확인
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # 프레임 단위로 이미지 저장할 디렉토리 생성
        video_output_directory = os.path.join(output_directory, filename[:-4])  # 확장자 제거
        if not os.path.exists(video_output_directory):
            os.makedirs(video_output_directory)

        frame_number = 0
        while frame_number < frame_count:
            ret, frame = cap.read()
            if not ret:
                break

            # 매 20프레임마다 이미지로 저장
            if frame_number % 3 == 0:
                image_name = f'{filename[:-4]}_frame_{frame_number:04d}.jpg'
                output_path = os.path.join(video_output_directory, image_name)
                cv2.imwrite(output_path, frame)

            frame_number += 1

        # 영상 파일의 작업 완료 후 리소스 해제
        cap.release()

cv2.destroyAllWindows()


# YOLOV5

## 사람 예측

In [ ]:
import os
import torch
import numpy as np
from PIL import Image

def normalize_coordinates(box, img_width, img_height):
    center_x = (box[0] + box[2]) / 2
    center_y = (box[1] + box[3]) / 2
    width = box[2] - box[0]
    height = box[3] - box[1]

    return center_x / img_width, center_y / img_height, width / img_width, height / img_height

# YOLOv5 모델 로드
model = torch.hub.load('ultralytics/yolov5', 'yolov5x')
model.conf = 0.6
model.iou = 0.5
model.classes = [0]  # 'person' class

# 이미지 폴더 경로와 결과를 저장할 폴더 경로
image_folder_path = '/content/drive/MyDrive/a/tr실험'
output_folder_path = '/content/drive/MyDrive/a/tr실험6'

# 폴더 내의 모든 이미지 파일에 대해서
for image_file in os.listdir(image_folder_path):
    image_path = os.path.join(image_folder_path, image_file)

    if os.path.isdir(image_path):
        continue

    pil_image = Image.open(image_path)
    img_width, img_height = pil_image.size
    im0s = np.array(pil_image)

    results = model(pil_image, size=640)
    det = results.xyxy[0].cpu()

    if det is not None:
        result_count = len(det)
    else:
        result_count = 0

    txt_file_name = os.path.splitext(image_file)[0] + '.txt'
    txt_file_path = os.path.join(output_folder_path, txt_file_name)

    if result_count == 0:
        # 객체가 감지되지 않은 경우 텍스트 파일을 삭제합니다.
        if os.path.exists(txt_file_path):
            os.remove(txt_file_path)
    else:
        with open(txt_file_path, 'w') as f:
            for i in range(result_count):
                class_id = int(det[i, 5].item())
                box = det[i, :4].tolist()
                normalized_coordinates = normalize_coordinates(box, img_width, img_height)

                f.write(f"{class_id} {' '.join(map(str, normalized_coordinates))}\n")

## truck 학습

In [ ]:
from glob import glob

img_list = glob('/content/drive/MyDrive/a/주행영상1_이미지처리/images/*.jpg')

print(len(img_list))

972


In [ ]:
from sklearn.model_selection import train_test_split

train_img_list, val_img_list = train_test_split(img_list, test_size=0.2, random_state=2000)

print(len(train_img_list), len(val_img_list))

777 195


In [ ]:
with open('/content/drive/MyDrive/a/주행영상1_이미지처리/train.txt', 'w') as f:
    f.write('\n'.join(train_img_list) + '\n')
with open('/content/drive/MyDrive/a/주행영상1_이미지처리/val.txt', 'w') as f:
    f.write('\n'.join(val_img_list) + '\n')

In [ ]:
import yaml

In [ ]:
with open('/content/drive/MyDrive/a/주행영상1_이미지처리/data.yaml', 'r')  as f:
    data = yaml.full_load(f)

print(data)

data['train'] = '/content/drive/MyDrive/a/주행영상1_이미지처리/train.txt'
data['val'] = '/content/drive/MyDrive/a/주행영상1_이미지처리/val.txtt'

with open('/content/drive/MyDrive/a/주행영상1_이미지처리/data.yaml', 'w') as f:
    yaml.dump(data, f)

print(data)

In [ ]:
%cat /content/drive/MyDrive/a/주행영상1_이미지처리/data.yaml

In [ ]:
%cd /content
!git clone https://github.com/ultralytics/yolov5.git

In [ ]:
%cd /content/yolov5/
!pip install -r requirements.txt

In [ ]:
%cd /content/yolov5/

!python train.py --img 640 --batch 7 --epochs 100 --data /content/drive/MyDrive/a/주행영상1_이미지처리/data.yaml --cfg ./models/yolov5x.yaml --weights yolov5x.pt --name /content/drive/MyDrive/a/truck_yolov5x_results1
# img 416 사이즈
# epoch 50번
# data는 yaml
# cfg는 모델의 구조(여기서는 s인 small 파일을 사용)
# weighhs는 pretrain된 yolov5s.pt를 다운받아서 사용
# name은 이름 설정

/content/yolov5
train: weights=yolov5x.pt, cfg=./models/yolov5x.yaml, data=/content/drive/MyDrive/빅웨이브_뉴론웍스/주행영상1_이미지처리/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=7, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=/content/drive/MyDrive/빅웨이브_뉴론웍스/truck_yolov5x_results1, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-217-g8c45e51 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_e

## truck 예측

In [ ]:
from glob import glob

img_list = glob('/content/drive/MyDrive/a/주행이미지4/labels/*.txt')

print(len(img_list))

474


In [ ]:
import os
import torch
import numpy as np
from PIL import Image

def normalize_coordinates(box, img_width, img_height):
    center_x = (box[0] + box[2]) / 2
    center_y = (box[1] + box[3]) / 2
    width = box[2] - box[0]
    height = box[3] - box[1]

    return center_x / img_width, center_y / img_height, width / img_width, height / img_height

# YOLOv5 모델 로드
model = torch.hub.load('ultralytics/yolov5', 'custom', '/content/drive/MyDrive/a/truck_yolov5x_results1/weights/best.pt')
model.conf = 0.3
model.iou = 0.5
model.classes = [0]  # 'person' class

# 이미지 폴더 경로와 결과를 저장할 폴더 경로
image_folder_path = '/content/drive/MyDrive/a/주행이미지4'
output_folder_path = '/content/drive/MyDrive/a/주행이미지4/labels'

# 폴더 내의 모든 이미지 파일에 대해서
for image_file in os.listdir(image_folder_path):
    image_path = os.path.join(image_folder_path, image_file)

    if os.path.isdir(image_path):
        continue

    pil_image = Image.open(image_path)
    img_width, img_height = pil_image.size
    im0s = np.array(pil_image)

    results = model(pil_image, size=640)
    det = results.xyxy[0].cpu()

    if det is not None:
        result_count = len(det)
    else:
        result_count = 0

    txt_file_name = os.path.splitext(image_file)[0] + '.txt'
    txt_file_path = os.path.join(output_folder_path, txt_file_name)

    if result_count == 0:
        # 객체가 감지되지 않은 경우 텍스트 파일을 삭제합니다.
        if os.path.exists(txt_file_path):
            os.remove(txt_file_path)
    else:
        with open(txt_file_path, 'w') as f:
            for i in range(result_count):
                class_id = int(det[i, 5].item())
                box = det[i, :4].tolist()
                normalized_coordinates = normalize_coordinates(box, img_width, img_height)

                f.write(f"{class_id} {' '.join(map(str, normalized_coordinates))}\n")

## 사람 txt와 truck txt 합치기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# 작업할 디렉토리 설정
folder_path = '/content/drive/MyDrive/a/주행이미지4/labels'  # 실제 폴더 경로로 변경

# 작업 디렉토리 내의 모든 텍스트 파일을 찾음
for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):  # .txt 확장자를 가진 파일만 처리
        file_path = os.path.join(folder_path, filename)

        # 파일 열기 및 읽기
        with open(file_path, 'r') as file:
            lines = file.readlines()

        # 각 줄의 첫 번째 열의 숫자를 1로 바꿈
        modified_lines = [f'1 {line.split(maxsplit=1)[1]}' for line in lines]

        # 파일 열기 및 쓰기 (원본 파일 덮어씀)
        with open(file_path, 'w') as file:
            file.writelines(modified_lines)

In [ ]:
import os
import shutil

folder_a = "/content/drive/MyDrive/a/주행이미지4/labels"  # a폴더의 경로를 여기에 입력하세요.
folder_b = "/content/drive/MyDrive/a/주행이미지4/사람label/주행4사람"  # b폴더의 경로를 여기에 입력하세요.

files_a = os.listdir(folder_a)
files_b = os.listdir(folder_b)

# a폴더와 b폴더에서 공통 파일 찾기
common_files = set(files_a) & set(files_b)

# a폴더에서 b폴더로 공통 파일을 추가
for filename in common_files:
    if filename.endswith(".txt"):
        file_path_a = os.path.join(folder_a, filename)
        file_path_b = os.path.join(folder_b, filename)

        # a폴더의 파일 내용 읽기
        with open(file_path_a, "r") as file_a:
            content_a = file_a.read()

        # b폴더의 파일 끝에 a폴더의 내용 추가
        with open(file_path_b, "a") as file_b:
            # a폴더의 내용을 b폴더의 파일에 추가하되, 줄바꿈으로 구분하여 추가
            file_b.write("\n" + content_a)

# a폴더에만 있는 파일을 b폴더로 복사
for filename in files_a:
    if filename.endswith(".txt") and filename not in common_files:
        file_path_a = os.path.join(folder_a, filename)
        file_path_b = os.path.join(folder_b, filename)

        # a폴더의 파일을 b폴더로 복사
        shutil.copy(file_path_a, file_path_b)

In [ ]:
import os

# 폴더 경로 a
folder_path = "/content/drive/MyDrive/a/주행이미지4/사람label/주행4사람"  # 실제 폴더 경로로 변경하세요.

# 폴더 내의 모든 .txt 파일 찾기
txt_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]

# 각 텍스트 파일에 대해 공백 줄 제거
for txt_file in txt_files:
    txt_file_path = os.path.join(folder_path, txt_file)

    # 파일 내용 읽어오기
    with open(txt_file_path, 'r') as file:
        lines = file.readlines()

    # 공백 줄 제거
    lines = [line.strip() for line in lines if line.strip()]

    # 수정된 내용을 파일에 다시 쓰기
    with open(txt_file_path, 'w') as file:
        file.writelines('\n'.join(lines))

print("모든 텍스트 파일에서 공백 줄을 제거하였습니다.")


모든 텍스트 파일에서 공백 줄을 제거하였습니다.
